In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fundus_vessels_toolkit import seg2graph, skeletonize
import cv2
import numpy as np

In [7]:
import jupyter_protopype as jpp
viewer = jpp.View2D()
viewer

View2D()

In [5]:
PATH = "15_dr"
raw = cv2.imread("/home/gaby/These/Data/Fundus/Vessels/HRF/0-images/"+PATH+".JPG")
vessels = cv2.imread("/home/gaby/These/Data/Fundus/Vessels/HRF/2-vessels/"+PATH+".tif")

def to_float(img):
    return img.astype(float)/255

raw = to_float(raw)
vessels = to_float(vessels).mean(axis=2) > .5
viewer.image = raw + np.expand_dims(vessels, 2)*.2


In [ ]:
def display(skel, skel_dist=None):
    bin_skel = skel>0
    raw_display = raw.copy()
    raw_display += np.expand_dims(vessels*.07, 2)
    if skel_dist is None:
        raw_display[bin_skel] = np.ones((3)) * .5
    else:
        raw_display[bin_skel] = np.expand_dims(skel_dist[bin_skel]/skel_dist.max(), 1)
    raw_display[skel==3] = [1,1,0]
    raw_display[skel==4] = [0,1,1]
    raw_display[skel==1] = [0,0,1]
    #raw_display[skel==2] = [1,1,1]
    #raw_display[skel==3] = [0,0,0]
    viewer.image = raw_display
    
def display_label(label, jonctions=None):
    import colorsys
    
    bin_skel = skel>0
    raw_display = raw.copy()
    raw_display += np.expand_dims(vessels*.07, 2)
    
    bin_label = np.tile(np.expand_dims(label!=0, 2), (1,1,3))
    colors = np.asarray([colorsys.hsv_to_rgb(h, 0.7,0.7) for h in np.linspace(0,1,30)])
    for i, c in enumerate(colors):
        raw_display[(label!=0) & (label%len(colors)==i)] = c
    if jonctions is not None:
        raw_display[jonctions] = [1,1,1]
    viewer.image = raw_display
viewer

In [ ]:
skel = skeletonize(vessels, return_distance=False, remove_small_ends=10)
display(skel)

In [ ]:
conn, branches, jonctions = seg2graph.extract_graph(skel_clean,return_label=True)
display_label(branches, jonctions)
conn.sum(axis=1)


In [ ]:
conn, branch_labels, idx = seg2graph.extract_graph(skel, return_label=True)
display_label(branch_labels, idx)

In [ ]:
conn.sum(axis=0).astype(int)

In [ ]:
conn.sum(axis=1).astype(int)

In [ ]:
np.sum(conn.sum(axis=1)>2)

In [ ]:
conn.sum(axis=1).shape